In [38]:
%cd '/home/guillaume/workspace/NewDeep/'
import numpy as np
import cPickle
import os
import sys
import theano
import theano.tensor as T
import utils
import nbests
import codecs
import collections
from joblib import Parallel, delayed
import multiprocessing

/home/guillaume/workspace/NewDeep


In [39]:
data_path = "/home/guillaume/Documents/CMU/cours/MT/git/sp2015.11-731/hw4/data"
bilingual_vectors_cs_path = "/home/guillaume/workspace/NewDeep/word_vectors/OutFileEE_orig1_projected.txt"
bilingual_vectors_en_path = "/home/guillaume/workspace/NewDeep/word_vectors/OutFileEE_orig2_projected.txt"

bilingual_vectors_cs_path = "/home/guillaume/workspace/NewDeep/word_vectors/cca.cs.50d.txt"
bilingual_vectors_en_path = "/home/guillaume/workspace/NewDeep/word_vectors/cca.en.50d.txt"

In [40]:
train_input_path = os.path.join(data_path, "train.input")
train_refs_path = os.path.join(data_path, "train.refs")

dev_test_input_path = os.path.join(data_path, "dev+test.input")
dev_refs_path = os.path.join(data_path, "dev.refs")

ttable_path = os.path.join(data_path, "ttable")

In [41]:
def read_vectors(filename):
    lines = [line.rstrip().split() for line in codecs.open(filename, 'r', "utf8")]
    if len(lines[0]) == 2:
        lines.pop(0)
    nb_words = len(lines)
    embedding_dimension = None
    words = []
    for i, line in enumerate(lines):
        if embedding_dimension == None:
            embedding_dimension = len(line) - 1
            vectors = np.zeros((nb_words, embedding_dimension))
        else:
            if not len(line) - 1 == embedding_dimension:
                print "Problem line %i.. skipped" % i
                continue
        words.append(line[0])
        vectors[i] = np.array([float(x) for x in line[1:]])
    return words, vectors

In [42]:
cs_words, cs_vectors = read_vectors(bilingual_vectors_cs_path)
print cs_vectors.shape
external_cs_voc = set(cs_words)
mapping_to_ext_cs = {}
for i, w in enumerate(cs_words):
    mapping_to_ext_cs[w] = i

(202089, 50)


In [43]:
en_words, en_vectors = read_vectors(bilingual_vectors_en_path)
print en_vectors.shape
external_en_voc = set(en_words)
mapping_to_ext_en = {}
for i, w in enumerate(en_words):
    mapping_to_ext_en[w] = i

(86330, 50)


In [44]:
train_input_lines = [[x.strip().split(" ") for x in line.rstrip().split("|||")] for line in codecs.open(train_input_path, 'r', "utf8")]
dev_test_input_lines = [[x.strip().split(" ") for x in line.rstrip().split("|||")] for line in codecs.open(dev_test_input_path, 'r', "utf8")]
for line in train_input_lines + dev_test_input_lines:
    assert len(line) == 3

dev_input_lines = dev_test_input_lines[:5791]
test_input_lines = dev_test_input_lines[5791:]

train_refs_lines = [line.rstrip() for line in open(train_refs_path, 'r')]
dev_refs_lines = [line.rstrip() for line in open(dev_refs_path, 'r')]

ttable_lines = [[x.strip().split(" ") for x in line.rstrip().split("|||")] for line in open(ttable_path, 'r')]

print "%i / %i / %i lines read from train / dev / ref inputs" % (len(train_input_lines), len(dev_input_lines), len(test_input_lines))
print "%i / %i lines read from train / dev refs" % (len(train_refs_lines), len(dev_refs_lines))
print "%i lines read from ttable" % len(ttable_lines)

122262 / 5791 / 5791 lines read from train / dev / ref inputs
122262 / 5791 lines read from train / dev refs
334770 lines read from ttable


In [45]:
def get_voc(lines):
    voc = {}
    for line in lines:
        for token in line[0] + line[1] + line[2]:
            if token in voc:
                voc[token] += 1
            else:
                voc[token] = 1
    return voc

In [49]:
train_english_tok_count = get_voc(train_input_lines)
dev_english_tok_count = get_voc(dev_input_lines)
test_english_tok_count = get_voc(test_input_lines)

train_english_tok_count = {k:v for k, v in train_english_tok_count.items() if k in external_en_voc}

train_english_tok_count["__EMPTY__"] = 0

train_english_voc = set(train_english_tok_count.keys())
dev_english_voc = set(dev_english_tok_count.keys())
test_english_voc = set(test_english_tok_count.keys())

ttable_tok_count = {}
for line in ttable_lines:
    for token in line[0]:
        if token in ttable_tok_count:
            ttable_tok_count[token] += 1
        else:
            ttable_tok_count[token] = 1
ttable_voc = set(ttable_tok_count.keys())

print "%i / %i / %i / %i different tokens in train / dev / test / ttable" % (len(train_english_voc), len(dev_english_voc), len(test_english_voc), len(ttable_voc))

46566 / 14648 / 14596 / 21460 different tokens in train / dev / test / ttable


In [50]:
"""for w, c in train_english_tok_count.items():
    if c < 20000 and not w.isdigit():
        print w"""

'for w, c in train_english_tok_count.items():\n    if c < 20000 and not w.isdigit():\n        print w'

In [51]:
word_to_index_en, index_to_word_en = utils.get_indexes(train_english_tok_count)

In [52]:
count = 0
sizes = {}
for line in ttable_lines:
    line = line[1]
    if len(line) not in sizes:
        sizes[len(line)] = 1
    else:
        sizes[len(line)] += 1
    count += 1
print count
print sizes

334770
{1: 198512, 2: 122291, 3: 12374, 4: 1344, 5: 208, 6: 26, 7: 5, 8: 4, 9: 1, 10: 4, 11: 1}


In [53]:
cs_voc_count = {}
for line in ttable_lines:
    assert len(line) == 3
    for token in line[1]:
        token = token.decode("utf8")
        if token in cs_voc_count:
            cs_voc_count[token] += 1
        else:
            cs_voc_count[token] = 1
word_to_index_cs, index_to_word_cs = utils.get_indexes(cs_voc_count)

ttable = {}
count_ignored = 0
for line in ttable_lines:
    assert len(line) == 3
    # if the word phrase is not in our dictionary
    should_break = False
    for token in line[0]:
        if token not in word_to_index_en:
            should_break = True
    if should_break:
        count_ignored += 1
        continue
    phrase_en = tuple([token.decode("utf8") for token in line[0]])
    phrase_cs = [token.decode("utf8") for token in line[1]]
    scores = np.array([float(x) for x in line[2]], dtype=np.float32)
    if phrase_en in ttable:
        ttable[phrase_en].append((phrase_cs, scores))
    else:
        ttable[phrase_en] = [(phrase_cs, scores)]

print "%i different phrases found in the phrase table for Czech, with %i in total." % (len(word_to_index_cs), sum(cs_voc_count.values()))
print "%i elements read from the phrase table, with %i different pairs of phrases." % (len(ttable), sum([len(x) for x in ttable.values()]))
print "%i phrases were ignored." % count_ignored

55237 different phrases found in the phrase table for Czech, with 486915 in total.
97371 elements read from the phrase table, with 331860 different pairs of phrases.
2910 phrases were ignored.


In [73]:
# every phrase we are interested in contains words in the dictionary
count_absent = 0
for i, (_, phrase, _) in enumerate(train_input_lines):
    for token in phrase:
        if not token in word_to_index_en:
            count_absent += 1
            print i, token, phrase
print "Absent in train: %i\n" % count_absent

count_absent = 0
for i, (_, phrase, _) in enumerate(dev_input_lines):
    for token in phrase:
        if not token in word_to_index_en:
            count_absent += 1
            print i, token, phrase
print "Absent in dev: %i\n" % count_absent

count_absent = 0
for i, (_, phrase, _) in enumerate(test_input_lines):
    for token in phrase:
        if not token in word_to_index_en:
            count_absent += 1
            print i, token, phrase
print "Absent in test: %i\n\n" % count_absent


# every phrase we are interested in is in the ttable
count_absent = 0
for i, (_, phrase, _) in enumerate(train_input_lines):
    if not tuple(phrase) in ttable:
        count_absent += 1
        print i, phrase
print "Absent in train: %i\n" % count_absent

count_absent = 0
for i, (_, phrase, _) in enumerate(dev_input_lines):
    if not tuple(phrase) in ttable:
        count_absent += 1
        print i, phrase
print "Absent in dev: %i\n" % count_absent

count_absent = 0
for i, (_, phrase, _) in enumerate(test_input_lines):
    if not tuple(phrase) in ttable:
        count_absent += 1
        print i, phrase
print "Absent in test: %i" % count_absent

Absent in train: 0

Absent in dev: 0

Absent in test: 0


Absent in train: 0

Absent in dev: 0

Absent in test: 0


In [72]:
train_input_lines = [l for i, l in enumerate(train_input_lines) if i != 34612]
dev_input_lines = [l for i, l in enumerate(dev_input_lines) if i != 356 and i != 5421]

In [74]:
count_absent = 0
for word in word_to_index_cs:
    if not word in external_cs_voc:
        count_absent += 1
print count_absent, len(word_to_index_cs)

127 55237


In [75]:
count_absent = 0
count_present = 0
sizes = {}
for _, phrase, _ in train_input_lines + dev_input_lines + test_input_lines:
    candidates = ttable[tuple(phrase)]
    for tokens in [candidate[0] for candidate in candidates]:
        if len(tokens) in sizes:
            sizes[len(tokens)] += 1
        else:
            sizes[len(tokens)] = 1
        for token in tokens:
            if not token in external_cs_voc:
                count_absent += 1
            else:
                count_present += 1
print count_absent
print count_present
print sizes

1281
5752152
{1: 4491698, 2: 541375, 3: 54611, 4: 3593, 5: 156}


In [175]:
threshold = 50000000

In [176]:
x_train = [[[x for x in l if x in word_to_index_en and (train_english_tok_count[x] < threshold or i == 1)] for i, l in enumerate(sentence)] for sentence in train_input_lines]
x_dev = [[[x for x in l if x in word_to_index_en and (train_english_tok_count[x] < threshold or i == 1)] for i, l in enumerate(sentence)] for sentence in dev_input_lines]
x_test = [[[x for x in l if x in word_to_index_en and (train_english_tok_count[x] < threshold or i == 1)] for i, l in enumerate(sentence)] for sentence in test_input_lines]

"""x_train = [(sentence[0] + sentence[2], sentence[1]) for sentence in x_train]
x_dev = [(sentence[0] + sentence[2], sentence[1]) for sentence in x_dev]
x_test = [(sentence[0] + sentence[2], sentence[1]) for sentence in x_test]

x_train = [[l if len(l) > 0 else [word_to_index_en["__EMPTY__"]] for l in sentence] for sentence in x_train]
x_dev = [[l if len(l) > 0 else [word_to_index_en["__EMPTY__"]] for l in sentence] for sentence in x_dev]
x_test = [[l if len(l) > 0 else [word_to_index_en["__EMPTY__"]] for l in sentence] for sentence in x_test]"""

print "%i / %i / %i elements in the train / dev / test." % (len(x_train), len(x_dev), len(x_test))

122261 / 5789 / 5791 elements in the train / dev / test.


In [177]:
countok = 0
countnok = 0
for i, (a, b, c) in enumerate(x_test):
    for w in b:
        if not w in external_en_voc:
            countnok += 1
        else:
            countok += 1
print countok, countnok

5821 0


In [178]:
for i, (a, b, c) in enumerate(x_test):
    print " ".join(a)
    print " ".join(b)
    print " ".join(c)
    print
    if i == 10:
        break

" there have even been
calls
to julian assange . "

said that assange was therefore
afraid
for his personal safety .

the wikileaks
founder
will remain in hiding .

was clearly reacting to statements from north
america
.

in america , the former republican governor of arkansas , mike , had called for the people
responsible
for the wikileaks revelations to be indicted and executed for treason .

in the meantime , exchanges have been made between the white
house
and assange .

robert , the speaker of american president barack obama , said that it was " ridiculous and absurd " that assange had demanded the resignation of secretary of state hillary clinton if she is found
responsible
for recently revealed requests to diplomats to engage in espionage .

" i 'm not entirely sure why we care about the
opinion
of a guy with a website , " said .

" our foreign policy and the interests of this country are far stronger than his one
website
. "

the american
government
is currently carrying out a 

In [179]:
count = 0
pb = 0
for a, b, c in x_test:
    if a == [word_to_index_en["__EMPTY__"]]:
        count += 1
    if c == [word_to_index_en["__EMPTY__"]]:
        count += 1
    assert b != [word_to_index_en["__EMPTY__"]]
print count

0


In [186]:
len(y_train)

122261

In [184]:
y_train = [phrase.decode("utf-8").split() for phrase in train_refs_lines]
y_train = [l for i, l in enumerate(y_train) if i != 34612]
y_dev = [phrase.decode("utf-8").split() for phrase in dev_refs_lines]
y_dev = [l for i, l in enumerate(y_dev) if i != 356 and i != 5421]

print "%i / %i references in the train / dev." % (len(y_train), len(y_dev))

122261 / 5789 references in the train / dev.


In [187]:
#######################
# Training parameters #
#######################
voc_size_en = len(index_to_word_en)
voc_size_cs = len(index_to_word_cs)

en_dim = 50
cs_dim = 50

k_top = 4

nb_filters = 4
filter_height = 6
filter_width = 2

layers_learning_rate = 0.01
embedding_learning_rate = 0.01

In [188]:
import layers
import networks
import conv_sentences

reload(layers)
reload(networks)
reload(conv_sentences)

############
# Training #
############

# inputs
context_input = T.imatrix(name='context_input')
phrase_input = T.imatrix(name='phrase_input')

# outputs
good_phrase = T.imatrix(name='phrase_good_output')
good_probs = T.fmatrix(name='phrase_good_probs')
bad_phrase = T.imatrix(name='phrase_bad_output')
bad_probs = T.fmatrix(name='phrase_bad_probs')

# is good
is_good = T.fvector(name="is_good")

# embeddings
embeddings_layer_en = layers.Embedding_layer(name='embedding_en', voc_size=voc_size_en, embedding_dim=en_dim, learning_rate=embedding_learning_rate)
embeddings_layer_cs = layers.Embedding_layer(name='embedding_cs', voc_size=voc_size_cs, embedding_dim=cs_dim, learning_rate=embedding_learning_rate)

# hidden layer
hidden_layer = layers.Biased_hidden_layer(name='hidden_layer', learning_rate=layers_learning_rate, input_dim=en_dim + 100 + 4, output_dim=10, activation=T.nnet.sigmoid)
#hidden_layer.weights.set_value(np.array(hidden_layer.weights.get_value(), dtype=np.float32))

# hidden layer 2
# dden_layer2 = layers.Biased_hidden_layer(name='hidden_layer_2', learning_rate=layers_learning_rate, input_dim=cs_dim, output_dim=1, activation=T.nnet.sigmoid)

# merge scores
merge_layer = layers.Biased_hidden_layer(name='merge_scores_layer', learning_rate=layers_learning_rate, input_dim=10, output_dim=1, activation=T.nnet.sigmoid)

In [189]:
vectors_values_en = embeddings_layer_en.weights.get_value()
for word in word_to_index_en:
    if word in mapping_to_ext_en:
        vectors_values_en[word_to_index_en[word]] = en_vectors[mapping_to_ext_en[word]]
embeddings_layer_en.weights.set_value(vectors_values_en)

vectors_values_cs = embeddings_layer_cs.weights.get_value()
for word in word_to_index_cs:
    if word in mapping_to_ext_cs:
        vectors_values_cs[word_to_index_cs[word]] = cs_vectors[mapping_to_ext_cs[word]]
embeddings_layer_cs.weights.set_value(vectors_values_cs)

In [190]:
# link
test_context = np.array(np.random.randint(0, 10, (11, 3)), dtype=np.int32)
test_phrase = np.array(np.random.randint(0, 10, (11, 2)), dtype=np.int32)

test_good_phrase = np.array(np.random.randint(0, 10, (11, 1)), dtype=np.int32)
test_bad_phrase = np.array(np.random.randint(0, 10, (11, 1)), dtype=np.int32)

test_good_probas = np.array(np.random.rand(11, 4), dtype=np.float32)
test_bad_probas = np.array(np.random.rand(11, 4), dtype=np.float32)

test_is_good = np.array(np.random.rand(11,), dtype=np.float32)

# we create a dimension that corresponds to a single feature map (en)
inputs_shape_en = T.join(0, [context_input.shape[0]], [context_input.shape[1]], [en_dim])
inputs_context = embeddings_layer_en.link([context_input])[0].reshape(inputs_shape_en, ndim=3)#.dimshuffle(0, 'x', 1, 2)

inputs_shape_phrase_en = T.join(0, [phrase_input.shape[0]], [phrase_input.shape[1]], [en_dim])
inputs_phrase = embeddings_layer_en.link([phrase_input])[0].reshape(inputs_shape_phrase_en, ndim=3)


print inputs_context.shape.eval({context_input:test_context})
print inputs_phrase.shape.eval({phrase_input:test_phrase})

inputs_context = T.mean(inputs_context, axis=1)
inputs_phrase = T.mean(inputs_phrase, axis=1)

print inputs_context.shape.eval({context_input:test_context})
print inputs_phrase.shape.eval({phrase_input:test_phrase})


# we create a dimension that corresponds to a single feature map (cs)
inputs_shape_cs = T.join(0, [good_phrase.shape[0]], [good_phrase.shape[1]], [cs_dim])
good_output = embeddings_layer_cs.link([good_phrase])[0].reshape(inputs_shape_cs, ndim=3)
good_output = T.mean(good_output, axis=1)
print good_output.shape.eval({good_phrase:test_good_phrase})



# compute the scores
norms = T.sqrt((inputs_context ** 2).sum(axis=1) * (good_output ** 2).sum(axis=1))
context_score = ((inputs_context * good_output).sum(axis=1) / norms).dimshuffle(0, 'x').repeat(100, axis=1)
print context_score.shape.eval({context_input:test_context, good_phrase:test_good_phrase})
#print context_score.repeat(10, axis=1).shape.eval({context_input:test_context, good_phrase:test_good_phrase})


#print good_score.shape.eval({context_input:test_context, good_phrase:test_good_phrase})
print good_probs.shape.eval({good_probs:test_good_probas})

#merged_good_scores = merge_layer.link([T.concatenate([good_output.dimshuffle(0, 'x'), good_probs], axis=1)])[0].flatten()
merged_good_scores = T.concatenate([context_score, inputs_phrase, good_probs], axis=1)

print merged_good_scores.shape.eval({context_input:test_context, phrase_input:test_phrase, good_phrase:test_good_phrase, good_probs:test_good_probas})


hidden_scores = hidden_layer.link([merged_good_scores])
print hidden_scores[0].shape.eval({context_input:test_context, phrase_input:test_phrase, good_phrase:test_good_phrase, good_probs:test_good_probas})


final_score = merge_layer.link(hidden_scores)[0].flatten()
print final_score.shape.eval({context_input:test_context, phrase_input:test_phrase, good_phrase:test_good_phrase, good_probs:test_good_probas})



cost = - T.mean(T.log( (1 - is_good) + (2 * is_good - 1) * final_score ))


print cost.eval({is_good:test_is_good, context_input:test_context, phrase_input:test_phrase, good_phrase:test_good_phrase, good_probs:test_good_probas})

#merged_bad_scores = theano.clone(merged_good_scores, {good_phrase:bad_phrase, good_probs:bad_probs})
#merged_bad_scores = merge_layer.link([bad_probs])[0].flatten()
#print merged_bad_scores.eval({left_input:test_l, right_input:test_r, phrase_input:test_p, bad_phrase:test_b, bad_probs:test_bp})


#final_score = (0.5 - merged_good_scores + merged_bad_scores)
#cost = (final_score * (final_score > 0)).mean()

#print final_score.eval({left_input:test_l, right_input:test_r, phrase_input:test_p, good_phrase:test_g, good_probs:test_gp, bad_phrase:test_b, bad_probs:test_bp})
#print final_score.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp, bad_phrase:test_b, bad_probs:test_bp})
#print cost.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp, bad_phrase:test_b, bad_probs:test_bp})
#print final_score.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp, bad_phrase:test_b, bad_probs:test_bp})
#print cost.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp, bad_phrase:test_b, bad_probs:test_bp})
#print final_score.eval({good_probs:test_gp,  bad_probs:test_bp})

#print merged_good_scores.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp})
#print cost.eval({left_input:test_l, right_input:test_r, good_phrase:test_g, good_probs:test_gp, is_good:test_ig})
#print cost.eval({good_probs:test_gp, is_good:test_ig})

[11  3 50]
[11  2 50]
[11 50]
[11 50]
[11 50]
[ 11 100]
[11  4]
[ 11 154]
[11 10]
[11]
0.733224511147


In [191]:
def updates(params, cost):
    gradients = T.grad(cost=cost, wrt=params)
    to_return = []
    for p, g in zip(params, gradients):
        learning_rate = embedding_learning_rate if str(p) == "embeddings" else layers_learning_rate
        to_return.append((p, p - learning_rate * g))
    return to_return

In [192]:
# build

params = merge_layer.params + hidden_layer.params + embeddings_layer_cs.params + embeddings_layer_en.params# + hidden_layer.params + conv_layer.params


func_feedforward = theano.function(
    inputs=[context_input, phrase_input, good_phrase, good_probs],
    outputs=final_score
)

func_train = theano.function(
    #inputs=[left_input, right_input, good_phrase, good_probs, bad_phrase, bad_probs],
    inputs=[context_input, phrase_input, good_phrase, good_probs, is_good],
    outputs=cost,
    updates=updates(params, cost)
)
"""
func_feedforward = theano.function(
    inputs=[good_probs],
    outputs=merged_good_scores
)

func_train = theano.function(
    inputs=[good_probs, is_good],
    outputs=cost,
    updates=updates(params, cost)
)
"""

'\nfunc_feedforward = theano.function(\n    inputs=[good_probs],\n    outputs=merged_good_scores\n)\n\nfunc_train = theano.function(\n    inputs=[good_probs, is_good],\n    outputs=cost,\n    updates=updates(params, cost)\n)\n'

In [193]:
func_feedforward(test_context, test_phrase, test_bad_phrase, test_bad_probas)
#func_feedforward(test_b, test_bp)

array([ 0.89995199,  0.89721417,  0.96563375,  0.98623151,  0.93535578,
        0.81931043,  0.89114976,  0.91361487,  0.88886243,  0.86775804,
        0.88542515], dtype=float32)

In [194]:
"""for i in xrange(10):
    if i % 1 == 0:
        print func_train(test_context, test_good_phrase, test_good_probas, test_is_good)
    else:
        pass
        #func_train(test_context, test_good_probas, test_bad, test_b)"""

'for i in xrange(10):\n    if i % 1 == 0:\n        print func_train(test_context, test_good_phrase, test_good_probas, test_is_good)\n    else:\n        pass\n        #func_train(test_context, test_good_probas, test_bad, test_b)'

In [173]:
layers_learning_rate = 0.03
embedding_learning_rate = 0.03

In [139]:
model_path = "/home/guillaume/Documents/CMU/cours/MT/git/sp2015.11-731/hw4/models/threshold20k_ctxt3_train_emb_epoch_%i_%s_%s_%f_%f"

In [ ]:
model_score_train, model_score_test = [0.527439, 0.397632]
model_epoch = 172789

hidden_layer.weights.set_value(cPickle.load(open(model_path % (model_epoch, "sh_first", "hidden_weights", model_score_train, model_score_test), "r")))
hidden_layer.bias.set_value(cPickle.load(open(model_path % (model_epoch, "sh_first", "hidden_bias", model_score_train, model_score_test), "r")))

merge_layer.weights.set_value(cPickle.load(open(model_path % (model_epoch, "sh_first", "merge_weights", model_score_train, model_score_test), "r")))
merge_layer.bias.set_value(cPickle.load(open(model_path % (model_epoch, "sh_first", "merge_bias", model_score_train, model_score_test), "r")))

In [ ]:
def pad(words, left, final_size=3):
    if len(words) == final_size:
        return words
    elif left:
        return word_to_index["__EMPTY__"] * (final_size - len(words)) + words
    else:
        return words + word_to_index["__EMPTY__"] * (final_size - len(words))

for epoch in xrange(20):
    j = 0
    all_costs = []
    recent_costs = []
    print "Starting epoch %i..." % epoch
    for i, (left, phrase, right) in enumerate(x_train):
        candidates = ttable[tuple(phrase)]
        """
        new_cost = func_train([left[-3:]], [right[:3]], [[y_train[i]]], [good_probas], [[candidates[j][0]]], [bad_probas])
        recent_costs.append(new_cost)
        all_costs.append(new_cost)
        """
        """good_probas = candidates[[x for x, _ in candidates].index(y_train[i])][1]
        
        #new_cost = func_train([left[-3:]], [right[:3]], [[y_train[i]]], [good_probas], np.array([1.0], dtype=np.float32))
        new_cost = func_train([good_probas], np.array([1.0], dtype=np.float32))
        recent_costs.append(new_cost)
        all_costs.append(new_cost)
        
        
        for _ in xrange(1):
            j = np.random.randint(len(candidates))
            while candidates[j][0] == y_train[i]:
                j = np.random.randint(len(candidates))
            bad_probas = candidates[j][1]
            #new_cost = func_train([left[-3:]], [right[:3]], [[candidates[j][0]]], [bad_probas], np.array([0.01], dtype=np.float32))
            new_cost = func_train([bad_probas], np.array([0.01], dtype=np.float32))
            recent_costs.append(new_cost)
            all_costs.append(new_cost)"""
        
        context = left[-4:] + right[:4]
        if len(context) == 0:
            context = ["__EMPTY__"]
        context = [word_to_index_en[x] for x in context]
        
        sizes = set([len(candidate[0]) for candidate in candidates])
        sorted_candidates = {size:[c for c in candidates if len(c[0]) == size] for size in sizes}
        
        for _, batch in sorted_candidates.items():
            #print [[word_to_index_cs[x] for x in b[0]] for b in batch]
            context_matrix = [context for _ in xrange(len(batch))]
            phrase_matrix = [context for _ in xrange(len(batch))]
            vector_is_good = np.array([1.0 if y_train[i] == b[0] else 0.001 for b in batch], dtype=np.float32)
            new_cost = func_train(context_matrix, phrase_matrix, [[word_to_index_cs[x] for x in b[0]] for b in batch], [b[1] for b in batch], vector_is_good)
            recent_costs.append(new_cost)
            all_costs.append(new_cost)
        
        if i % 10000 == 0:
            print "%i, average cost since last time: %f" % (i, np.mean(recent_costs))
            recent_costs = []
    
        if i % 50000 == 0:
            positions_train = Parallel(n_jobs=num_cores)(delayed(compute_position_train)(i) for i in xrange(len(x_dev)))
            positions_dev = Parallel(n_jobs=num_cores)(delayed(compute_position_dev)(i) for i in xrange(len(x_dev)))
            score_train, score_dev = compute_final_score(positions_train), compute_final_score(positions_dev)
            print score_train, score_dev
    
    print "Epoch %i done. Average cost of this epoch: %f" % (epoch, np.mean(all_costs))
    positions_train = Parallel(n_jobs=num_cores)(delayed(compute_position_train)(i) for i in xrange(len(x_dev)))
    positions_dev = Parallel(n_jobs=num_cores)(delayed(compute_position_dev)(i) for i in xrange(len(x_dev)))
    score_train, score_dev = compute_final_score(positions_train), compute_final_score(positions_dev)
    print score_train, score_dev
    cPickle.dump(hidden_layer.weights.get_value(), open(model_path % (epoch, "sh_second", "hidden_weights", score_train, score_dev), "w"))
    cPickle.dump(hidden_layer.bias.get_value(), open(model_path % (epoch, "sh_second", "hidden_bias", score_train, score_dev), "w"))
    cPickle.dump(merge_layer.weights.get_value(), open(model_path % (epoch, "sh_second", "merge_weights", score_train, score_dev), "w"))
    cPickle.dump(merge_layer.bias.get_value(), open(model_path % (epoch, "sh_second", "merge_bias", score_train, score_dev), "w"))

In [96]:
layers_learning_rate = 0.01
embedding_learning_rate = 0.01

In [97]:
num_cores = 7 #multiprocessing.cpu_count()

In [195]:
def compute_position(x, y):
    left, phrase, right = x
    good = y
    candidates = ttable[tuple(phrase)]
    left = [left for i in xrange(len(candidates))]
    phrase = [phrase for i in xrange(len(candidates))]
    right = [right for i in xrange(len(candidates))]
    scores = np.array(func_feedforward(left, right, [[z[0]] for z in candidates], [z[1] for z in candidates]))
    #scores = np.array(func_feedforward([z[1] for z in candidates]))
    #scores = np.array(func_feedforward([z[1] for z in candidates]))
    return (list(np.argsort(scores))[::-1].index([x for x, _ in candidates].index(good)) + 1)

def compute_position(x, y):
    left, phrase, right = x
    good = y
    candidates = ttable[tuple(phrase)]
    
    context = left[-4:] + right[:4]
    if len(context) == 0:
        context = ["__EMPTY__"]
    context = [word_to_index_en[x] for x in context]
    phrase = [word_to_index_en[x] for x in phrase]

    sizes = set([len(candidate[0]) for candidate in candidates])
    sorted_candidates = {size:[c for c in candidates if len(c[0]) == size] for size in sizes}
    all_scores = []
    
    for _, batch in sorted_candidates.items():
        context_matrix = [context for _ in xrange(len(batch))]
        phrase_matrix = [phrase for _ in xrange(len(batch))]
        #vector_is_good = np.array([1.0 if y_train[i] == b[0] else 0.001 for b in batch], dtype=np.float32)
        scores = func_feedforward(context_matrix, phrase_matrix, [[word_to_index_cs[x] for x in b[0]] for b in batch], [b[1] for b in batch])
        all_scores.extend([(tuple(tok), score) for tok, score in zip([b[0] for b in batch], scores)])
    #print all_scores
    #return
    return [x[0] for x in sorted(all_scores, key=lambda x: x[1], reverse=True)].index(tuple(good)) + 1
    #return (list(np.argsort(scores))[::-1].index([x for x, _ in candidates].index(tuple(good))) + 1)

def compute_position_train(i):
    return compute_position(x_train[i], y_train[i])

def compute_position_dev(i):
    return compute_position(x_dev[i], y_dev[i])

def compute_best_words(x):
    left, phrase, right = x
    candidates = ttable[tuple(phrase)]
    
    context = left[-3:] + right[3:]
    if len(context) == 0:
        context = ["__EMPTY__"]
    context = [word_to_index_en[x] for x in context]
    phrase = [word_to_index_en[x] for x in phrase]

    sizes = set([len(candidate[0]) for candidate in candidates])
    sorted_candidates = {size:[c for c in candidates if len(c[0]) == size] for size in sizes}
    all_scores = []
    
    for _, batch in sorted_candidates.items():
        context_matrix = [context for _ in xrange(len(batch))]
        phrase_matrix = [phrase for _ in xrange(len(batch))]
        scores = func_feedforward(context_matrix, phrase_matrix, [[word_to_index_cs[x] for x in b[0]] for b in batch], [b[1] for b in batch])
        all_scores.extend([(tuple(tok), score) for tok, score in zip([b[0] for b in batch], scores)])
    #print all_scores
    #return
    
    #return [" ".join(x[0]) for x in sorted(all_scores, key=lambda x: x[1], reverse=True)]
    return " ||| ".join([" ".join(x[0]) for x in sorted(all_scores, key=lambda x: x[1], reverse=True)])

def compute_words_dev(i):
    if i < 356:
        return compute_best_words(x_dev[i])
    elif i == 356:
        return u"orbáne ||| orbán ||| orbána ||| orbánovi"
    elif i > 356 and i < 5421:
        return compute_best_words(x_dev[i-1])
    elif i == 5421:
        return u"spolupracovali ||| spolupracovala ||| spolupracovaly ||| spolupracoval"
    else:
        return compute_best_words(x_dev[i-2])

def compute_words_test(i):
    return compute_best_words(x_test[i])

In [259]:
output_dev_file_path = "/home/guillaume/Documents/CMU/cours/MT/git/sp2015.11-731/hw4/output_dev_1.txt"
with open(output_dev_file_path, "w") as f:
    f.write(u"\n".join([compute_words_dev(i) for i in xrange(len(x_dev)+1)]).encode("utf8"))
    f.write(u"\n")
    f.write(u"\n".join([compute_words_test(i) for i in xrange(len(x_test))]).encode("utf8"))

In [258]:
len(x_test)

5791

In [48]:
compute_position_dev(13)

1

In [102]:
aaa = merge_layer.weights.get_value()

In [49]:
[x[0] for x in merge_layer.weights.get_value()]

[1.1638297,
 -0.65674418,
 -2.0892873,
 -1.4943478,
 0.90839487,
 1.2587247,
 1.552141,
 1.1953601,
 1.7400082,
 0.69266123]

In [148]:
values = [1, 0, 0.3, 0]
values = [2.07229996, 0.7576499, -0.2130859, 0.24015519, 0.18058282]
values = [2.07229996, 0, 0, 0, 0]
merge_layer.weights.set_value(np.array([[x] for x in values], dtype=np.float32))

In [148]:
positions_train = Parallel(n_jobs=num_cores)(delayed(compute_position_train)(i) for i in xrange(len(x_dev)+1))
positions_dev = Parallel(n_jobs=num_cores)(delayed(compute_position_dev)(i) for i in xrange(len(x_dev)))

In [101]:
def compute_final_score(positions):
    return np.mean([1.0 / x for x in positions])

In [162]:
compute_final_score(positions_train)

0.65728945775343628

In [163]:
compute_final_score(positions_dev)

0.4075099062044697

In [149]:
compute_final_score(positions_train)

0.57807477901939808

In [150]:
compute_final_score(positions_dev)

0.41191242927781341

In [135]:
positions = []
for x, y in zip(x_dev, y_dev):
    candidates = [z[0] for z in sorted(ttable[tuple(x[1])], key=lambda y: np.sum(y[1]), reverse=True)]
    positions.append(candidates.index(y) + 1)

In [137]:
final_score(positions)

0.37160985215792669

In [560]:
cs_wvectors = cPickle.load(open("/home/guillaume/workspace/NewDeep/word_vectors/polyglot-cs.pkl", "r"))
en_wvectors = cPickle.load(open("/home/guillaume/workspace/NewDeep/word_vectors/polyglot-en.pkl", "r"))

In [ ]:
/home/guillaume/workspace/NewDeep/word_vectors

In [565]:
len(cs_wvectors[1])

100004

In [973]:
en_wvectors[1]

array([[-0.24006841,  0.0170402 , -0.5328812 , ..., -0.22872141,
         0.42429936, -1.06828964],
       [ 0.04196003,  0.12553459, -0.10587557, ..., -0.22909568,
         0.3757889 ,  0.24626437],
       [ 0.1696005 ,  1.13571692,  0.93620867, ..., -0.68687445,
         0.35930997, -0.01402457],
       ..., 
       [-0.02400243,  0.02764406,  0.11886688, ...,  0.05557791,
        -0.20100911,  0.09282091],
       [-0.22458398,  0.23377183,  0.15628465, ...,  0.26328474,
        -0.17130157, -0.32867849],
       [-0.03621942, -0.28310931, -0.01432774, ..., -0.10810753,
        -0.37851715,  0.22238818]], dtype=float32)

In [167]:
cs_vectors.shape

(202089, 50)

In [164]:
def find_closests(word, english=True):
    vector = en_vectors[mapping_to_ext_en[word]]
    best_indexes_en = np.argsort(en_vectors.dot(vector))[::-1][:10]
    best_indexes_cs = np.argsort(cs_vectors.dot(vector))[::-1][:10]
    for i in best_indexes_en:
        print en_words[i]
    for i in best_indexes_cs:
        print cs_words[i]

In [166]:
find_closests("dead")

dead
dying
nameless
kilter
ixtapaluca
destroyed
thrown
ignites
humanoids
died
hácích
poseta
nezemřela
vstal
zachráníme
proletí
střelen
mrtvá
drezínu
pustém


In [161]:
[[x for x in l if x in word_to_index_en and (train_english_tok_count[x] < threshold or i == 1)] for i, l in enumerate(dev_input_lines[357])]

[[u'gauge', u'situation', u'proportion'],
 [u'believe'],
 [u'dissatisfaction',
  u'prime',
  u'minister',
  u'managed',
  u'without',
  u'having',
  u'put',
  u'government',
  u'office']]